In [ ]:
from torch.utils.data import Dataset

In [ ]:
class ChbDataset(Dataset):
    def __init__(self, list_IDs, labels):
        'Initialization'
        pass
        
    def __len__(self):
        'Denotes the total number of samples'
        pass
        
    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        pass